In [1]:
import numpy as np
import cv2
from collections import defaultdict
import time


this code is to make kalman depend on the state for acceleration,velocity and position in x and y direction

In [7]:
block = 3

#this function will generate the vicinity near its area to check whether there is any near point
def dist_of_points(prev_point,new_point,block_size = block):
    #prev_points are trace points
    #new_points ate new feature points
    ind = np.indices((block_size,block_size))
    
    (px,py) = prev_point
    (nx,ny) = new_point
    if block_size%2==0:
        print("it is divisible by 2")
        return False
    c = (block_size+1)/2
    #this code contain the row and col value w.r.t the prev_point as a centre
    row = np.arange((1-c),abs(1-c)+1)#rows w-r-t the prev points
    for i in range(1,block_size+1):
        #col w-r-t the prev points
        col = i-c
        for j in row:
            #print(j,col)
            if px+col==nx and py+j==ny:
                return True
    return False
def euclidean_dist(prev_point,new_point,epsilon=0.3):
    (px,py) = prev_point
    (nx,ny) = new_point
    square_x =(nx-px)**2
    square_y =(ny-py)**2
    a = np.sqrt(square_x+square_y)
    if a<epsilon:
        return True
    return False

In [8]:
#kalman filter
def k_predict(x,P,FPS=30,delta_t=1):
    #x is state [x,y,vx,vy,ax,ay]
    #p is uncertainity covariance
    #F = state transistion matrix
    F = np.array([[1,0,delta_t,0,delta_t**2,0],
                  [0,1,0,delta_t,0,delta_t**2],
                  [0,0,1,0,delta_t,0],
                  [0,0,0,1,0,delta_t],
                  [0,0,0,0,1,0],
                  [0,0,0,0,0,1]])
    x_pred = np.dot(F,x)
    p_pred = np.dot(np.dot(F,P),F.T)
    return x_pred,p_pred

def k_update(z,x_pred,p_pred,delta_t=1):
    R = np.identity(6)*10#measurement uncertainity assume to be 1 size depend upon the the H rows
    I = np.identity(6) #identity matrix size depend upon the P_pred row
    H = np.array([[1,0,delta_t,0,delta_t**2,0],
                  [0,1,0,delta_t,0,delta_t**2],
                  [0,0,1,0,delta_t,0],
                  [0,0,0,1,0,delta_t],
                  [0,0,0,0,0,1],
                  [0,0,0,0,0,1]])#measurment function contain all the information velocity as well as position in x and y coord
    y = z - np.dot(H,x_pred)#residul or error
    s = np.dot(np.dot(H,p_pred),H.T)+R
    k = np.dot(np.dot(p_pred,H.T),np.linalg.inv(s))#kalman gain
    x_next = x_pred+np.dot(k,y) #corrected state
    p_next = np.dot((I-np.dot(k,H)),p_pred) #corrected covariance
    return x_next,p_next

In [4]:
#step1
#feature parameter for the shi tomasi algo
feature_params = dict(maxCorners=1,
                    qualityLevel=0.2,
                    minDistance = 10,
                    blockSize = 10)

#parameters for lucasKanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#create the color
#color  = np.random.randint(0,255,(300,3))

check_id = 0 #it is the id given to each object
main_id = 0
object_ids = dict() #it is the dictionary which can contains the id and the coordinate
disappeared = dict() #it is the dictionary which contains the number of time the object is dissappeared
max_disappeared = 10 #till 10 frames the object will be followed up if it is not visible after 10 frames it will be remove

max_points = 30
points = defaultdict(list)

cap = cv2.VideoCapture('H:/swaayyat_robots/optical_flow/dataset/optical_flow_test.mp4')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#create the feature points
fp = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#state initialization
px,py = fp.ravel()
state = np.array([px,py,0,0,0,0]).reshape(6,1) #[x,y,vx,vy]
#P matrix
cov = np.array([[1,0,1,0,1,0],
                [0,1,0,1,0,1],
                [0,0,1,0,1,0],
                [0,0,0,1,0,1],
                [0,0,0,0,1,0],
                [0,0,0,0,0,1]])*10
#print(fp)
#prev velocities
v_prev_x = 0 
v_prev_y=0
#deltaT is same as delta_t
deltaT=1
while(cap.isOpened()):
    ret,frame = cap.read()
    if ret==True:
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        tp,st,err = cv2.calcOpticalFlowPyrLK(old_gray,gray_frame,fp,None,**lk_params)
        good_new = tp[st==1]
        good_old = fp[st==1]
        image = frame.copy()
        image1 = image.copy()
        image2 = image.copy()
        
        for i,(old,new) in enumerate(zip(good_old,good_new)):
            a,b = old.ravel().astype(int)
            c,d = new.ravel().astype(int)
            vx = c-a
            vy = c-a
            z = np.array([c,d,vx,vy,(vx-v_prev_x)/deltaT,(vy-v_prev_y)/deltaT]).reshape(6,1)
            try:
                main_id = [key for key,values in object_ids.items() if values[0]==a and values[1]==b][0]
            except Exception as e:
                main_id=-1
                pass

            if main_id<0:
                check_id+=1
                object_ids[check_id] = object_ids.get(check_id,(c,d))
                disappeared[check_id] = disappeared.get(check_id,0)
                points[check_id].append((c,d))
                main_id=check_id
            else:
                object_ids[main_id]=(c,d)
                points[main_id].append((c,d))
                if len(points[main_id])>max_points:
                    points[main_id].pop(0)
            for newp,point in enumerate(points[main_id]):
                if newp>=len(points[main_id])-2:
                    break
                (rpx,rpy) = point
                (rnx,rny) = points[main_id][newp+1]
                image = cv2.line(frame,(rpx,rpy),(rnx,rny),(0,255,0),2)
            image = cv2.circle(image,(c,d),2,(0,255,0),2)
            image1 = image.copy()
            image1 = cv2.putText(image1,"x:"+str(c),(c-2,d-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
            
            for iterate in range(20):
                x_predict,p_predict = k_predict(state,cov)
                x_next,p_next = k_update(z,x_predict,p_predict)
                pfpx = int(round(x_predict[0][0]))
                pfpy = int(round(x_predict[1][0]))
                pfpvx = int(x_predict[2][0])
            image = cv2.circle(image,(pfpx,pfpy),2,(0,0,255),2)
            image2 = image.copy()
            image2 = cv2.putText(image2,"x:"+str(pfpx),(pfpx-2,pfpy-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
            state = x_next
            cov = p_next
            v_prev_x = vx
            v_prev_y = vy
            
        #new_fp = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
        #for nfp in new_fp:
        #    (nfpx,nfpy) = nfp.ravel()
        #    image = cv2.circle(image,(nfpx,nfpy),2,(255,0,0),2)
        fp = good_new.reshape(-1,1,2)
        old_gray = gray_frame.copy()
        cv2.imshow('frame',frame)
        cv2.imshow('opticalflow',image)
        cv2.imshow('original',image1)
        cv2.imshow('kalman',image2)
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()


In [10]:
# step2 for multiple points
#feature parameter for the shi tomasi algo
feature_params = dict(maxCorners=10,
                    qualityLevel=0.2,
                    minDistance = 10,
                    blockSize = 10)

#parameters for lucasKanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#create the color
#color  = np.random.randint(0,255,(300,3))

check_id = 0 #it is the id given to each object
main_id = 0
object_ids = dict() #it is the dictionary which can contains the id and the coordinate
disappeared = dict() #it is the dictionary which contains the number of time the object is dissappeared
max_disappeared = 10 #till 10 frames the object will be followed up if it is not visible after 10 frames it will be remove
#state and noise initialization for n corner points
state = defaultdict(np.array)
P_cov = defaultdict(np.array)

max_points = 30
points = defaultdict(list)
points_kalman = defaultdict(list)

cap = cv2.VideoCapture('H:/swaayyat_robots/optical_flow/dataset/optical_flow_test.mp4')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#create the feature points
fp = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#Velocity
v_prev = defaultdict(tuple)

#state initialization
for i,fppp in enumerate(fp):
    (c,d) = fppp.ravel()
    check_id+=1
    object_ids[check_id] = object_ids.get(check_id,(c,d))
    disappeared[check_id] = disappeared.get(check_id,0)
    points[check_id].append((c,d))
    points_kalman[check_id].append((c,d))
    state[check_id] = np.array([c,d,0,0,0,0]).reshape(6,1) #[x,y,vx,vy]
    P_cov[check_id] = np.array([[1,0,1000,0,1000,0],
                      [0,1,0,1000,0,1000],
                      [0,0,1000,0,1000,0],
                      [0,0,0,1000,0,1000],
                      [0,0,0,0,1000,0],
                      [0,0,0,0,0,1000]])
    v_prev[check_id] = (0,0) #vx,vy
  
    #print(fp)
#deltaT is delta_t 
deltaT=1
while(cap.isOpened()):
    ret,frame = cap.read()
    if ret==True:
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        tp,st,err = cv2.calcOpticalFlowPyrLK(old_gray,gray_frame,fp,None,**lk_params)
        good_new = tp[st==1]
        good_old = fp[st==1]
        image = frame.copy()
        image1 = image.copy()
        image2 = image.copy()

        for i,(old,new) in enumerate(zip(good_old,good_new)):
            a,b = old.ravel().astype(int)
            c,d = new.ravel().astype(int)
            
            try:
                main_id = [key for key,values in object_ids.items() if values[0]==a and values[1]==b][0]
            except Exception as e:
                main_id=-1
                pass

            if main_id<0:
                check_id+=1
                object_ids[check_id] = object_ids.get(check_id,(c,d))
                disappeared[check_id] = disappeared.get(check_id,0)
                points[check_id].append((c,d))
                main_id=check_id
            else:
                object_ids[main_id]=(c,d)
                points[main_id].append((c,d))
                if len(points[main_id])>max_points:
                    points[main_id].pop(0)
            for newp,point in enumerate(points[main_id]):
                if newp>=len(points[main_id])-2:
                    break
                (rpx,rpy) = point
                (rnx,rny) = points[main_id][newp+1]
                image = cv2.line(frame,(rpx,rpy),(rnx,rny),(255,0,0),2)
            image1 = cv2.circle(image1,(c,d),2,(255,0,0),2)
            image1 = cv2.putText(image1,"x:"+str(c),(c-2,d-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
            #measurement
            vx = c-a
            vy = d-b
            v_prev_x = v_prev[main_id][0]
            v_prev_y = v_prev[main_id][1]
            ax = (vx-v_prev_x)/deltaT
            ay = (vy-v_prev_y)/deltaT
            z = np.array([c,d,vx,vy,ax,ay]).reshape(6,1)
            #for iterate in range(10):
            x_predict,p_predict = k_predict(state[main_id],P_cov[main_id],delta_t=deltaT)
            x_next,p_next = k_update(z,x_predict,p_predict,delta_t=deltaT)
            pfpx = int(x_predict[0][0])
            pfpy = int(x_predict[1][0])
            pfpvx = int(x_predict[2][0])
            pfpvy = int(x_predict[3][0])
            #pfpx = int(x_next[0][0])
            #pfpy = int(x_next[1][0])
            
            image1 = cv2.circle(image1,(pfpx,pfpy),2,(0,0,255),2)
            image1 = cv2.putText(image1,"x:"+str(pfpx),(pfpx-2,pfpy-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
            state[main_id] = x_next
            P_cov[main_id] = p_next
        #new_fp = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
        #for nfp in new_fp:
        #    (nfpx,nfpy) = nfp.ravel()
        #    image = cv2.circle(image,(nfpx,nfpy),2,(255,0,0),2)
        fp = good_new.reshape(-1,1,2)
        old_gray = gray_frame.copy()
        #cv2.imshow('frame',frame)
        #cv2.imshow('main',image)
        cv2.imshow('original',image1)
        #cv2.imshow('kalman',image2)
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()

In [11]:
#step 3
#tracking the  new feature as well final code 
#feature parameter for the shi tomasi algo
feature_params = dict(maxCorners=50,
                    qualityLevel=0.4,
                    minDistance = 10,
                    blockSize = 10)

#parameters for lucasKanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#create the color
#color  = np.random.randint(0,255,(300,3))
block = 3 #block size for computing the near pixels

check_id = 0 #it is the id given to each object
main_id = 0
object_ids = dict() #it is the dictionary which can contains the id and the coordinate
#disappeared = dict() #it is the dictionary which contains the number of time the object is dissappeared
max_disappeared = 10 #till 10 frames the object will be followed up if it is not visible after 10 frames it will be remove

#n = time.time()
cap = cv2.VideoCapture('H:/swaayyat_robots/optical_flow/dataset/optical_flow_test.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

#state and noise initialization for n corner points
state = defaultdict(np.array)
P_cov = defaultdict(np.array)

max_points = 30
points = defaultdict(list)
points_kalman = defaultdict(list)

cap = cv2.VideoCapture('H:/swaayyat_robots/optical_flow/dataset/optical_flow_test.mp4')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#create the feature points
fp = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#Velocity
v_prev = defaultdict(tuple)

#state initialization
for i,fppp in enumerate(fp):
    (c,d) = fppp.ravel()
    check_id+=1
    object_ids[check_id] = object_ids.get(check_id,(c,d))
    disappeared[check_id] = disappeared.get(check_id,0)
    points[check_id].append((c,d))
    points_kalman[check_id].append((c,d))
    state[check_id] = np.array([c,d,0,0,0,0]).reshape(6,1) #[x,y,vx,vy]
    P_cov[check_id] = np.array([[1,0,1000,0,10000,0],
                      [0,1,0,1000,0,10000],
                      [0,0,1000,0,10000,0],
                      [0,0,0,1000,0,10000],
                      [0,0,0,0,10000,0],
                      [0,0,0,0,0,10000]])
    v_prev[check_id] = (0,0) #vx,vy
  
    #print(fp)
#deltaT is delta_t 
deltaT=1

while(cap.isOpened()):
    ret,frame = cap.read()
    if ret==True:
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        tp,st,err = cv2.calcOpticalFlowPyrLK(old_gray,gray_frame,fp,None,**lk_params)
        good_new = tp[st==1]
        good_old = fp[st==1]
        update = np.copy(good_new)
        image1 = frame.copy()
        image2 = frame.copy()
        new_fp = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
        new_fp = new_fp.reshape(-1,2)
        #idx_memory=0
        #cnt=0
        
        for i,(old,new) in enumerate(zip(good_old,good_new)):
            a,b = old.ravel().astype(int)
            c,d = new.ravel().astype(int)
            
            for idx,nfp in enumerate(new_fp):
                (nfpx,nfpy) = nfp.ravel()        
                result = dist_of_points((c,d),(nfpx,nfpy),block_size=block)
                if result==True:
                    new_fp = np.delete(new_fp,idx,0)
                    break
                
            if (a,b) not in object_ids.values():
                if (c,d) not in object_ids.values():
                    check_id+=1
                    object_ids[check_id] = object_ids.get(check_id,(c,d))
                    #disappeared[check_id] = disappeared.get(check_id,0)
                    points[check_id].append((c,d))
                    v_prev[check_id] = (0,0)
                    state[check_id] = np.array([c,d,0,0,0,0]).reshape(6,1) #[x,y,vx,vy]
                    P_cov[check_id] = np.array([[1,0,1000,0,1000,0],
                                              [0,1,0,1000,0,1000],
                                              [0,0,1000,0,1000,0],
                                              [0,0,0,1000,0,1000],
                                              [0,0,0,0,1000,0],
                                              [0,0,0,0,0,1000]])
                    main_id=check_id
                else:
                    main_id = [key for key,values in object_ids.items() if values[0]==c and values[1]==d][0]
            else:
                main_id = [key for key,values in object_ids.items() if values[0]==a and values[1]==b][0]
                
                object_ids[main_id]=(c,d)
                points[main_id].append((c,d))
                
                if len(points[main_id])>max_points:
                    points[main_id].pop(0)
            
            for newp,point in enumerate(points[main_id]):
                if newp>=len(points[main_id])-2:
                    break
                (rpx,rpy) = point
                (rnx,rny) = points[main_id][newp+1]
                image1 = cv2.line(image1,(rpx,rpy),(rnx,rny),(255,0,0),2)
            image1 = cv2.circle(image1,(c,d),2,(255,0,0),2)
    
            
            #measurement
            vx = c-a
            vy = d-b
            v_prev_x = v_prev[main_id][0]
            v_prev_y = v_prev[main_id][1]
            ax = (vx-v_prev_x)/deltaT
            ay = (vy-v_prev_y)/deltaT
            z = np.array([c,d,vx,vy,ax,ay]).reshape(6,1)
            #for iterate in range(10):
            x_predict,p_predict = k_predict(state[main_id],P_cov[main_id],delta_t=deltaT)
            x_next,p_next = k_update(z,x_predict,p_predict,delta_t=deltaT)
            pfpx = int(x_predict[0][0])
            pfpy = int(x_predict[1][0])
            pfpvx = int(x_predict[2][0])
            pfpvy = int(x_predict[3][0])
            points_kalman[main_id].append((pfpvx,pfpvy))
            #pfpx = int(x_next[0][0])
            #pfpy = int(x_next[1][0])
            #for newpk,pointk in enumerate(points_kalman[main_id]):
            #    if newpk>=len(points_kalman[main_id])-2:
            #        break
            #    (krpx,krpy) = pointk
            #    (krnx,krny) = points_kalman[main_id][newpk+1]
            #    image2 = cv2.line(image2,(krpx,krpy),(krnx,krny),(0,0,255),2)
            image2 = cv2.circle(image2,(pfpx,pfpy),2,(0,0,255),2)
            #image2 = cv2.putText(image2,"x:"+str(pfpx),(pfpx-2,pfpy-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
            state[main_id] = x_next
            P_cov[main_id] = p_next
            
            
        for ffp in new_fp:
            (ffpx,ffpy) = ffp.ravel()
            update = np.append(update,[[ffpx,ffpy]],0)
            
        fp = update.reshape(-1,1,2)
        old_gray = gray_frame.copy()
        #cv2.imshow('frame',frame)
        cv2.imshow('original_trace',image1)
        cv2.imshow('kalman predicted',image2)
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()


In [25]:
H

array([[1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1]])